In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, SGDRegressor
from sklearn.pipeline import make_pipeline

DATASET_PATH = "datasets/"

def load_song_data(dataset_path=DATASET_PATH):
    df = pd.read_csv(dataset_path + 'year-prediction-msd-train.txt', sep=",", header=None)
    return df

In [2]:
def iter_minibatches(chunksize):
    chunkstartmarker = 0
    while chunkstartmarker < len(songs):
        x_chunk = songs_x_train[chunkstartmarker:chunkstartmarker+chunksize]
        y_chunk = songs_y_train[chunkstartmarker:chunkstartmarker+chunksize]
        yield x_chunk, y_chunk
        chunkstartmarker += chunksize

In [3]:
#songs é um DataFrame
songs = load_song_data()

In [4]:
songs_x = songs.iloc[:,1:]
songs_y = songs.iloc[:,0]
#songs_x = songs_x.values.reshape(-1,1)
songs_y = songs_y.values.reshape(-1,1)

In [5]:
#Normalizing
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])
prepared = num_pipeline.fit_transform(songs_x)

In [6]:
songs_x_train, songs_x_test, songs_y_train, songs_y_test = train_test_split(prepared, songs_y, test_size=0.045, random_state=0)

In [7]:
for count, degree in enumerate([2]):
    batcherator = iter_minibatches(chunksize=1000)
    regr = make_pipeline(PolynomialFeatures(degree), SGDRegressor(learning_rate='invscaling', eta0=0.001))
    
    for x_chunk, y_chunk in batcherator:
        if(len(x_chunk)!=0):
            regr.named_steps['sgdregressor'].partial_fit(x_chunk, y_chunk)
    
songs_y_pred = regr.named_steps['sgdregressor'].predict(songs_x_test)

/home/fagner/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
# The coefficients
print('Estimated intercept: ', regr.named_steps['sgdregressor'].intercept_)

# The coefficients
print('Coefficients: ', regr.named_steps['sgdregressor'].coef_)

# The mean squared error
print("Mean squared error: %.2f"
      % np.sqrt(mean_squared_error(songs_y_test, songs_y_pred)))

Estimated intercept:  [ 1998.36809877]
Coefficients:  [  5.21635179e+00  -2.58996451e+00  -8.06871084e-01  -1.43174272e+00
  -2.51864464e-01  -2.21656930e+00   4.57697231e-01  -9.47092540e-01
  -7.97800451e-01   5.03296440e-01   1.95373359e-02  -2.36290953e-01
   9.72118683e-01   6.83633188e-01  -5.81697055e-01   9.20728055e-01
   2.74740859e-01   1.35931281e+00   6.99227357e-01   1.14300888e+00
   2.28575025e-01  -5.10982687e-01   1.31414919e+00   3.96205717e-01
  -3.66364354e-01   1.47981970e-02   8.51233606e-01   6.93601151e-02
   2.26815623e-01   5.53554745e-02  -9.73407778e-02  -6.35737876e-02
  -4.02517187e-01   1.08071356e-01   2.60245802e-03  -5.47963263e-01
  -4.27300392e-02   2.39362745e-01   4.95011989e-01  -3.84879964e-01
  -3.07272590e-01  -9.45212175e-02  -1.10642976e-01  -1.04262042e-01
  -9.19168058e-02   2.15181674e-01   1.69788222e-01  -5.91478576e-01
   8.25773296e-02   2.46620520e-01  -8.52669774e-03  -1.66072789e-01
   1.54388149e-01  -9.15542296e-02   5.48206220e-